In [1]:
import pandas as pd
import numpy as np

In [2]:
reward = pd.read_csv('./reward.csv',encoding='utf-8')

In [3]:
reward.shape

(24024943, 5)

In [4]:
reward

,account_id,product_id,purchase,book_cluster,user_cluster
0,100001,88440267,1.0,3.0,3.0
1,100001,88440267,1.0,3.0,3.0
2,100001,85951536,1.0,3.0,3.0
3,100001,85951536,1.0,3.0,3.0
4,100001,86895523,0.0,3.0,3.0
...,...,...,...,...,...
24024938,16837205,65060443,1.0,0.0,2.0
24024939,16837205,42763617,1.0,0.0,2.0
24024940,16837207,84678826,1.0,3.0,2.0
24024941,16837207,22791986,0.0,0.0,2.0


In [ ]:
context = pd.read_csv('./full_context.csv',encoding='utf-8')

In [ ]:
context.shape

In [ ]:
reward

In [ ]:
context

In [43]:
context_1000 = context.iloc[:1000,1:]

In [44]:
reward_1000 = reward.iloc[:1000]
reward_1000 = reward_1000['purchase']

In [311]:
narms = 5
ndims = 72
v = 0.9

b = np.identity(ndims)
B = np.vstack((b,b,b,b,b))
mu_hat = np.zeros((ndims,narms))
f = np.zeros(ndims)
F = np.vstack((f,f,f,f,f))

arm_number = reward['book_cluster'].iloc[:1200].astype('int')

bounds = np.zeros((ndims,narms))

In [240]:
arm_number[1]

3

In [241]:
i = 1
j = arm_number[i]

In [242]:
mean = np.transpose(mu_hat)[j]
covariance_matrix = ((v)**2)*np.linalg.inv(B[ndims*j:ndims*j+ndims])

In [247]:
mean.shape

(72,)

In [246]:
covariance_matrix.shape

(72, 72)

In [249]:
samples = np.random.multivariate_normal(mean, covariance_matrix)+np.random.randn(ndims)
sample_mu_tilde = np.expand_dims(samples,axis=1)

In [253]:
sample_mu_tilde.shape

(72, 1)

In [254]:
arm_context = np.expand_dims(np.transpose(context_1000.iloc[i]),axis=1)

In [256]:
arm_context.shape

(72, 1)

In [257]:
B[ndims*j:ndims*j+ndims] = B[ndims*j:ndims*j+ndims] + np.dot(arm_context,np.transpose(arm_context))

In [281]:
B[72:144].shape

(72, 72)

In [284]:
np.squeeze(arm_context).shape

(72,)

In [271]:
arm_context.shape

(72, 1)

In [285]:
F[j] = F[j] + np.squeeze(arm_context)*reward_1000[i]

In [286]:
F[j]

array([0.        , 0.34051455, 0.        , 0.        , 0.        ,
       0.        , 0.31689489, 0.        , 0.        , 0.        ,
       0.        , 0.13609702, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.68345648,
       0.        , 0.23642498, 0.        , 0.        , 0.        ,
       0.11008415, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.17141852, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.36621018, 0.        ])

In [291]:
np.dot(np.linalg.inv(B[ndims*j:ndims*j+ndims]),F[j]).shape

(72,)

In [298]:
mu_hat.shape

(72, 5)

In [361]:
narms = 5
ndims = 72
v = 0.9

b = np.identity(ndims)
B = np.vstack((b,b,b,b,b))
mu_hat = np.zeros((ndims,narms))
f = np.zeros(ndims)
F = np.vstack((f,f,f,f,f))

arm_number = reward['book_cluster'].iloc[:1200].astype('int')

bounds = np.zeros(narms)

In [378]:
for i in context_1000.index:
    
    j = arm_number[i]
        
    mean = np.transpose(mu_hat)[j]
    covariance_matrix = ((v)**2)*np.linalg.inv(B[ndims*j:ndims*j+ndims])
    
    samples = np.random.multivariate_normal(mean, covariance_matrix)+np.random.randn(ndims)
    sample_mu_tilde = np.expand_dims(samples,axis=1)
    
    arm_context = np.expand_dims(np.transpose(context_1000.iloc[i]),axis=1)
    bounds[j] = bounds[j] + np.dot(np.transpose(arm_context),sample_mu_tilde)[0][0]
    
    
    B[ndims*j:ndims*j+ndims] = B[ndims*j:ndims*j+ndims] + np.dot(arm_context,np.transpose(arm_context))
    F[j] = F[j] + np.squeeze(arm_context)*reward_1000[i]
    np.transpose(mu_hat)[j] = np.dot(np.linalg.inv(B[ndims*j:ndims*j+ndims]),F[j])

In [395]:
bounds

array([ 26.91224399,  55.85019365,  -3.1322682 , 106.50657819,
        69.36519875])

In [396]:
bounds/1000

array([ 0.02691224,  0.05585019, -0.00313227,  0.10650658,  0.0693652 ])

In [389]:
r1000 = reward.iloc[:1000]
r1000 = r1000[['purchase','book_cluster']]

In [398]:
r1000.groupby('book_cluster').sum()/1000

,purchase
book_cluster,
0.0,0.036
1.0,0.115
2.0,0.023
3.0,0.111
4.0,0.049


In [321]:
test = context.iloc[1000:1100,1:]

In [322]:
test

,gender_0.0,gender_1.0,g_age_0.0,g_age_1.0,g_age_2.0,g_age_3.0,g_age_4.0,g_age_5.0,g_age_6.0,most_pref_1.0,...,cat34.0,pub0.0,pub1.0,pub2.0,pub3.0,pub4.0,pri0.0,pri1.0,pri2.0,pri3.0
1000,0.666066,0.0,0.0,0.0,0.0,0.0,0.396425,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.236211,0.000000,0.0,0.263816,0.000000,0.0
1001,0.666066,0.0,0.0,0.0,0.0,0.0,0.396425,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.327535,0.0,0.263816,0.000000,0.0
1002,0.666066,0.0,0.0,0.0,0.0,0.0,0.396425,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.236211,0.000000,0.0,0.263816,0.000000,0.0
1003,0.659485,0.0,0.0,0.0,0.0,0.0,0.316895,0.0,0.0,0.0,...,0.0,0.0,0.129505,0.0,0.000000,0.000000,0.0,0.000000,0.371458,0.0
1004,0.659485,0.0,0.0,0.0,0.0,0.0,0.316895,0.0,0.0,0.0,...,0.0,0.0,0.129505,0.0,0.000000,0.000000,0.0,0.263816,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,0.666066,0.0,0.0,0.0,0.0,0.0,0.396425,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.232846,0.000000,0.0,0.264633,0.000000,0.0
1096,0.666066,0.0,0.0,0.0,0.0,0.0,0.396425,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.232846,0.000000,0.0,0.264633,0.000000,0.0
1097,0.666066,0.0,0.0,0.0,0.0,0.0,0.396425,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.232846,0.000000,0.0,0.264633,0.000000,0.0
1098,0.666066,0.0,0.0,0.0,0.0,0.0,0.396425,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.324797,0.0,0.264633,0.000000,0.0


In [323]:
i = 10001
j = arm_number[1001]

In [393]:
j

1

In [324]:
mean = np.transpose(mu_hat)[j]
covariance_matrix = ((v)**2)*np.linalg.inv(B[ndims*j:ndims*j+ndims])

In [335]:
B[72:144]

array([[107.474554  ,   0.        ,   0.        , ...,  21.43466274,
          0.        ,   0.        ],
       [  0.        ,   5.2374517 ,   0.        , ...,   3.17456827,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   1.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [ 21.43466274,   3.17456827,   0.        , ...,  13.17564431,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          1.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   1.        ]])

In [339]:
F[1]

array([6.92708686e+01, 3.67327352e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.50176688e+01, 1.22891748e+01, 1.38662617e-01,
       1.04440086e-02, 0.00000000e+00, 0.00000000e+00, 1.22661071e+00,
       0.00000000e+00, 7.66128684e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 6.87915259e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.03464416e+00, 0.00000000e+00, 8.17316811e+00, 1.84876146e-01,
       0.00000000e+00, 0.00000000e+00, 1.06693666e+02, 1.42133946e-01,
       2.21252948e+00, 7.99413194e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.07425906e-01, 6.24312506e-02,
      

In [343]:
mu_hat.T[1]

array([ 0.10169725,  0.47638799,  0.        ,  0.        ,  0.        ,
        0.17634052, -0.19862435,  0.0627501 ,  0.00403337,  0.        ,
        0.        ,  0.22956046, -0.00675947,  0.02958709, -0.00477388,
        0.        ,  0.        ,  0.        , -0.00317188,  0.        ,
        0.        ,  0.        ,  0.00514136,  0.        , -0.00753831,
        0.        ,  0.        , -0.07178677,  0.08078577,  0.        ,
        0.08491339,  0.13148385, -0.27093344,  0.        ,  0.21110643,
        0.06432372,  0.27115088,  0.35921174,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.15974377,
        0.03378969, -0.1284057 ,  0.06565097,  0.        , -0.41108745,
       -0.1170032 ,  0.02917303, -0.00450911,  0.        , -0.04113469,
        0.2749041 , -0.05948939, -0.19839395,  0.68805035, -0.66

In [325]:
mean

array([ 0.10169725,  0.47638799,  0.        ,  0.        ,  0.        ,
        0.17634052, -0.19862435,  0.0627501 ,  0.00403337,  0.        ,
        0.        ,  0.22956046, -0.00675947,  0.02958709, -0.00477388,
        0.        ,  0.        ,  0.        , -0.00317188,  0.        ,
        0.        ,  0.        ,  0.00514136,  0.        , -0.00753831,
        0.        ,  0.        , -0.07178677,  0.08078577,  0.        ,
        0.08491339,  0.13148385, -0.27093344,  0.        ,  0.21110643,
        0.06432372,  0.27115088,  0.35921174,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.15974377,
        0.03378969, -0.1284057 ,  0.06565097,  0.        , -0.41108745,
       -0.1170032 ,  0.02917303, -0.00450911,  0.        , -0.04113469,
        0.2749041 , -0.05948939, -0.19839395,  0.68805035, -0.66

In [326]:
covariance_matrix

array([[ 0.20541315,  0.2569249 ,  0.        , ...,  0.00916099,
         0.        ,  0.        ],
       [ 0.2569249 ,  0.53648529, -0.        , ..., -0.03641934,
        -0.        , -0.        ],
       [ 0.        ,  0.        ,  0.81      , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.00916099, -0.03641934,  0.        , ...,  0.26709913,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.81      ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.81      ]])

In [327]:
samples = np.random.multivariate_normal(mean, covariance_matrix)+np.random.randn(ndims)
sample_mu_tilde = np.expand_dims(samples,axis=1)

In [329]:
sample_mu_tilde.shape

(72, 1)

In [399]:
arm_context = np.expand_dims(np.transpose(test.iloc[1]),axis=1)

In [348]:
B[ndims*j:ndims*j+ndims] = B[ndims*j:ndims*j+ndims] + np.dot(arm_context,np.transpose(arm_context))
F[j] = F[j] + np.squeeze(arm_context)*reward.iloc[1001][2]
np.transpose(mu_hat)[j] = np.dot(np.linalg.inv(B[ndims*j:ndims*j+ndims]),F[j])

In [350]:
reward.iloc[1001][2]

0.0

In [349]:
np.dot(np.transpose(arm_context),sample_mu_tilde)

array([[-0.04016789]])

In [353]:
B[72:144]

array([[108.80548592,   0.        ,   0.        , ...,  21.96181844,
          0.        ,   0.        ],
       [  0.        ,   5.2374517 ,   0.        , ...,   3.17456827,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   1.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [ 21.96181844,   3.17456827,   0.        , ...,  13.38444021,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          1.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   1.        ]])

In [354]:
F[1]

array([6.92708686e+01, 3.67327352e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.50176688e+01, 1.22891748e+01, 1.38662617e-01,
       1.04440086e-02, 0.00000000e+00, 0.00000000e+00, 1.22661071e+00,
       0.00000000e+00, 7.66128684e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 6.87915259e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.03464416e+00, 0.00000000e+00, 8.17316811e+00, 1.84876146e-01,
       0.00000000e+00, 0.00000000e+00, 1.06693666e+02, 1.42133946e-01,
       2.21252948e+00, 7.99413194e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.07425906e-01, 6.24312506e-02,
      

In [357]:
mu_hat.T[1]

array([ 0.0930866 ,  0.47662776,  0.        ,  0.        ,  0.        ,
        0.18317585, -0.20955348,  0.06292453,  0.00401889,  0.        ,
        0.        ,  0.2467448 , -0.00661838,  0.0294809 , -0.00484855,
        0.        ,  0.        ,  0.        , -0.00309018,  0.        ,
        0.        ,  0.        ,  0.00510926,  0.        , -0.00724499,
        0.        ,  0.        , -0.07150118,  0.08407032,  0.        ,
        0.09553267,  0.13405108, -0.33661854,  0.        ,  0.21716392,
        0.06344399,  0.27010536,  0.3532111 ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.16204211,
        0.03492834, -0.12783222,  0.06838065,  0.        , -0.4101927 ,
       -0.11109931, -0.00566553, -0.00317627,  0.        , -0.04070396,
        0.28110759, -0.07575798, -0.18875318,  0.69556003, -0.66